In [2]:
import glob
import re
import pandas as pd
from pandas import DataFrame
from yattag import Doc
import unicodedata

In [3]:
doc, tag, text = Doc().tagtext()
txt_file = glob.glob('AmostrasDODF/**/*.txt')

all_contracts = []
all_text = ''

for row in range(len(txt_file)):
    instance = re.findall(r'[\s\S]+', 
                unicodedata.normalize('NFKD', open(txt_file[row], encoding='utf-8').read()))

    if txt_file[row] == txt_file[0]:
        all_text += instance[0] + '\n'
    else:
        if txt_file[row][:-7] == txt_file[row-1][:-7]:
            all_text += instance[0] + '\n'
            if txt_file[row] == txt_file[-1]:   
                all_contracts.append(txt_file[row][:-7] + '\n' + all_text)
        else:
            all_contracts.append(txt_file[row-1][:-7] + '\n' + all_text)
            all_text = ''
            all_text += instance[0] + '\n'


dodfs = []

for contract in all_contracts:
    dodfs.append(contract.split('\n'))

In [4]:
len(dodfs)

20

## Função DFA

In [12]:
def dfa_licitacao():
    regex5 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+RESULTADO\s+D[EO]\s+JULGAMENTO|AVISO\s+D[EO]\s+JULGAMENTO)'
    regex6 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+LICITAÇÃO)'
    regex7 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+JULGAMENTO\s+D[EO]\s+HABILITAÇÃO)'
    regex8 = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+RESULTADO\s+D[EO]\s+RECURSO\s+E\s+JULGAMENTO)'

    regex_s = r'xxbcet (?:AVISOS?|EXTRATOS?|RESULTADOS?|SECRETARIA|PREGÃO|TOMADA|COMISSÃO)'


    acts = []
    l_acts = []
    ato = False

    for texto in dodfs:
        i = 0
        # print(len(texto))
        while i != len(texto):
            if re.match(regex5, texto[i]):
                l_acts.append('[5] ' + texto[0] + texto[i])
                ato = True
                while ato:
                    i += 1
                    if i == len(texto):
                        break
                    if not re.match(regex5, texto[i]) and re.match(regex_s, texto[i]) and ('xxbob' in texto[i-1] or('—' in texto[i-1] and 'xxbob' in texto[i-2])):
                        i -= 2
                        break
                    elif re.match(regex5, texto[i]):
                        i -= 2
                        break
                    else:
                        l_acts[-1] += '\n' + texto[i]

            elif re.match(regex6, texto[i]):
                l_acts.append('[6] ' + texto[0] + texto[i])
                ato = True
                while ato:
                    i += 1
                    if i == len(texto):
                        break
                    if not re.match(regex6, texto[i]) and re.match(regex_s, texto[i]) and ('xxbob' in texto[i-1] or('—' in texto[i-1] and 'xxbob' in texto[i-2])):
                        i -= 1
                        break
                    elif re.match(regex6, texto[i]):
                        i -= 1
                        break
                    else:
                        l_acts[-1] += '\n' + texto[i]

            elif re.match(regex7, texto[i]):
                l_acts.append('[7] ' + texto[0] + texto[i])
                ato = True
                while ato:
                    i += 1
                    if i == len(texto):
                        break
                    if not re.match(regex7, texto[i]) and re.match(regex_s, texto[i]) and ('xxbob' in texto[i-1] or('—' in texto[i-1] and 'xxbob' in texto[i-2])):
                        i -= 1
                        break
                    elif re.match(regex7, texto[i]):
                        i -= 1
                        break
                    else:
                        l_acts[-1] += '\n' + texto[i]

            elif re.match(regex8, texto[i]):
                l_acts.append('[8] ' + texto[0] + texto[i])
                ato = True
                while ato:
                    i += 1
                    if i == len(texto):
                        break
                    if not re.match(regex8, texto[i]) and re.match(regex_s, texto[i]) and ('xxbob' in texto[i-1] or('—' in texto[i-1] and 'xxbob' in texto[i-2])):
                        i -= 1
                        break
                    elif re.match(regex8, texto[i]):
                        i -= 1
                        break
                    else:
                        l_acts[-1] += '\n' + texto[i]

            else:
                i+=1

    return l_acts


In [18]:
acts = dfa_licitacao()

In [19]:
len(acts)

87

In [21]:
acts[0]

'[6] AmostrasDODF\\DODF 008 11-01-2012 SECAO3_txt\\xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃO ELETRÔNICO No 8/2012. xxecet\nACOMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO FEDERAL torna público\nque realizará o Pregão Eletrônico PE-008/2012-CAESB, processo no 092.011281/2011, Aquisi-\nção de Reagentes Químicos e Vidrarias, conforme especificações técnicas constantes no Anexo\nI(Pedido de Aquisição/Termo de Referência), por preço equalizado, por lote cotado. Data final\nxxeob\n\nxxbob\npara Recebimento das Propostas: 26 de janeiro de 2012, às 08h30. Início da Sessão de Disputa:\n26 de janeiro 2012, às 10h00. O edital e seus anexos poderão ser encontrados no site www.\nlicitacoes-e.com.br a partir do dia 12 de janeiro de 2012. Outras informações poderão ser obtidas\npor meio do telefone (61) 3213-7275, email pregao(dcaesb.df.gov.br.\nxxbcet Brasília/DF, 10 de janeiro de 2012. xxecet\nxxbcet RAULINDO JUNIOR NAVES REZENDE xxecet\nxxbcet Pregoeiro Caesb xxecet

In [22]:
df = DataFrame(acts,columns=['Texto'])

# Regex para encontrar o id do ato
df['Id_ato'] =  df['Texto'].apply(lambda st: st[1:2])
re_exp = r'\[5\]|\[6\]|\[7\]|\[8\]'
df['Texto'] =  df['Texto'].str.replace(re_exp,'')


# Regex para encontrar o dodf de onde está sendo retirado a informação
df['Dodf'] =  df['Texto'].apply(lambda st: st.split('txt\\')[0])
df['Texto'] =  df['Texto'].apply(lambda st: st.split('txt\\')[1])


# Regex para encontrar o processo do ato
df['Processo'] = df['Texto'].str.extract(r'''(?:(?:(?:P|p)rocesso(?:\s+)?(?:(?:\()?SEI(?:\)?))?(?:\s+)?(?:(?:no|n\.o)?)?)|(?:P|p)rocesso:|(?:P|p)rocesso|Processo.|(?:P|p)rocesso\s+no|(?:P|p)rocesso\s+n.? ?o.?|(?:P|p)rocesso\s+no:|(?:P|p)rocesso\s+SEI\s+no:|(?:P|p)rocesso\s+SEI:|(?:P|p)rocesso\s+SEI-GDF:|(?:P|p)rocesso\s+SEI-GDF|(?:P|p)rocesso\s+SEI\s+no|(?:P|p)rocesso\s+SEI\s+n|(?:P|p)rocesso\s+SEI|(?:P|p)rocesso-\s+SEI|(?:P|p)rocesso\s+SEI\s+no.|(?:P|p)rocesso\s+\(SEI\)\s+no.|(?:P|p)rocesso\s+SEI\.|(?:P|p)rocesso\s+\(SEI-DF\)\s+no.|(?:P|p)rocesso\s+SEI-GDF no|(?:P|p)rocesso\s+n|(?:P|p)rocesso\s+N|(?:P|p)rocesso\s+administrativo no|(?:P|p)rocesso\s+n:|PROCESSO: ?N?o?|PROCES-? ?SO|PROCESSO.|PROCESSO\s+no|PROCESSO\s+No|PROCESSO\s+N.o:?|PROCESSO\s+no.|PROCESSO\s+no:|PROCESSO\s+No:|PROCESSO\s+SEI\s+no:|PROCESSO\s+SEI:|PROCESSO\s+SEI|PROCESSO\s+SEI-GDF:|PROCESSO\s+SEI-GDF|PROCESSO\s+SEI\s+no|PROCESSO\s+SEI\s+No|PROCESSO\s+SEI\s+no.|PROCESSO\s+SEI.)((?:(?!\s\d{2}.\d{3}.\d{3}/\d{4}-\d{2}))(?:(?:\s)(?:(?:[\d.]+)|(?:[\d\s]+))[.-]?(?:(?:\d)|(?:[.\d\sSEI-|]+))(?:/|-
\b)(?:(?:(?:\d)+|(?:[\d\s]+)))?(?:-(?:(?:\d)+|(?:[\d\s]+)))?(?:-SECOM/DF|-?CEB|/CBMDF|F J Z B / D F)?))''')


# Regex para encontrar a data do ato
df['Data_dodf'] = df['Texto'].str.extract(r'''Brasí?i?lia(?:\/?DF)?,?\s+(\d{2}\s+de+\s\w+\s+de\s+\d{4})''')
for row in range(len(df)):
    if pd.isna(df['Data_dodf'][row]):
        df['Data_dodf'][row] = re.findall(r'\d{2}-\d{2}-\d{4}', df['Dodf'][row])[0]

In [23]:
df

,Texto,Id_ato,Dodf,Processo,Data_dodf
0,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PRE...,6,AmostrasDODF\DODF 008 11-01-2012 SECAO3_,092.011281/2011,10 de janeiro de 2012
1,xxbcet AVISO DO RESULTADO DE JULGAMENTO xxecet...,5,AmostrasDODF\DODF 008 11-01-2012 SECAO3_,060.004.060/2011,10 de janeiro de 2012
2,xxbcet AVISO DO RESULTADO DE JULGAMENTO DESERT...,5,AmostrasDODF\DODF 008 11-01-2012 SECAO3_,\n411.000.335/2010,10 de janeiro de 2012
3,xxbcet AVISO DE LICITAÇÃO DESERTA xxecet\nxx...,6,AmostrasDODF\DODF 008 11-01-2012 SECAO3_,411.000.075/2011,10 de janeiro de 2011
4,xxbcet AVISO DE LICITAÇÃO NOVA DATA xxecet\n...,6,AmostrasDODF\DODF 008 11-01-2012 SECAO3_,NaN,10 de janeiro de 2012
...,...,...,...,...,...
82,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet TOM...,6,AmostrasDODF\DODF 209 30-10-2007 SECAO3_,137.000.385/\n2007,26 de outubro de 2007
83,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet CON...,6,AmostrasDODF\DODF 209 30-10-2007 SECAO3_,NaN,29 de outubro de 2007
84,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet CON...,6,AmostrasDODF\DODF 209 30-10-2007 SECAO3_,00092.004.823/2007,29 de outubro de 2007
85,xxbcet AVISO DE RESULTADO DE JULGAMENTO xxecet...,5,AmostrasDODF\DODF 229 06-12-2005 SECAO3_,052.000.922/2004,05 de dezembro de 2005


In [24]:
df.isnull().sum()

Texto         0
Id_ato        0
Dodf          0
Processo     23
Data_dodf     0
dtype: int64

In [25]:
df['Texto'][72]

'xxbcet AVISO DE JULGAMENTO DE LICITAÇÃO xxecet\nxxbcet TOMADA DE PREÇOS No 07/2010. xxecet\nO PRESIDENTE DA COMISSÃO PERMANENTE DE LICITAÇÕES, DA COMPANHIA\nDE SANEAMENTO AMBIENTAL DO DISTRITO FEDERAL, TORNA PÚBLICO o\nresultado do julgamento da Tomada de Preços no 07/2010-CAESB, processo 092.001.810/\n2010, tipo de licitação: menor preço, para contratação de empresa especializada para serviços\nde projeto, fabricação e fornecimento de 01 (um) Centro de Controle de Motores (CCM) para\na Elevatória Booster do Lago Sul, EBO.LS.02, do Sistema de Abastecimento de Água da Caesb,\nna forma de execução indireta, sob regime de empreitada por preço global, da forma que se\nsegue: a empresa AG TECH SISTEMAS LTDA - EPP julgada vencedora com o valor global de\nR$ 80.000,00 (oitenta mil reais).\nxxbcet Brasília/DF, 27 de agosto de 2010 xxecet\nxxbcet JERÔNIMO BAREICHA JÚNIOR xxecet\nxxeob\n\nxxbob'

## Texto com as tags de marcação

In [30]:
# data = df[['Texto']]
# data

In [31]:
# data.to_csv('CSVs/atos_with_tags.csv')

## Texto sem as tags de marcação

In [32]:
# re_exp = r'xxbcet ?|xxbcet ?|xxeob ?|xxbob ?|xxecet ?'
# data['Texto'] =  data['Texto'].str.replace(re_exp,'')
# data

In [33]:
# data.to_csv('CSVs/atos_without_tags.csv')